In this notebook, we will demonstrate the concept of ensemble learning. With ensemble models, we combine multiple models together to create a more accurate prediction.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Load the dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)





In [3]:
# Pipeline: Integrate preprocessing into the training process
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', DecisionTreeClassifier())
])

# Number of bootstrap samples and trees in the ensemble
n_samples = len(X_train)
n_trees = 50

def bootstrap_sample(X, y):
    indices = np.random.choice(len(X), size=len(X), replace=True)
    return X[indices], y[indices]



In [4]:
# Train multiple decision trees on different bootstrap samples
trees = []
for _ in range(n_trees):
    X_sample, y_sample = bootstrap_sample(X_train, y_train)
    
    # Clone the pipeline for each tree
    cloned_pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('clf', DecisionTreeClassifier())])
    cloned_pipeline.fit(X_sample, y_sample)
    trees.append(cloned_pipeline)



In [8]:
trees

[Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassifier())]),
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('clf', DecisionTreeClassif

In [5]:
# Aggregate predictions
predictions = np.zeros((len(X_test), n_trees))
for tree_index, tree in enumerate(trees):
    predictions[:, tree_index] = tree.predict(X_test)



In [9]:
predictions

array([[1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [2., 2., 2., ..., 2., 2., 2.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
# Majority vote
y_pred = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=predictions.astype('int'))



In [7]:
# Test accuracy
print(f"Accuracy (Custom Bagged Trees): {accuracy_score(y_test, y_pred)*100:.2f}%")

Accuracy (Custom Bagged Trees): 100.00%
